In [1]:
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
from torchvision.transforms import v2

In [2]:
SET_ROOT = './data'
NUM_CHANNELS = 3 # RGB format
NUM_CLASSES_CIFAR10 = 10 # In CIFAR10 Dataset
EPOCHS = 71

In [3]:
transf_test = v2.Compose([
    v2.RandomHorizontalFlip(),
    v2.RandomVerticalFlip(),
    v2.ToTensor()
])
transf_train = v2.Compose([
    v2.ToTensor()
])

testset = CIFAR10(root=SET_ROOT, 
                  train=False, 
                  download=True, 
                  transform=transf_test)

trainset = CIFAR10(root=SET_ROOT, 
                   train=True, 
                   download=True, 
                   transform=transf_train)

print(f"Len test: {len(testset)}")
print(f"Len train: {len(trainset)}")

C:\Users\hanne\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified
Len test: 10000
Len train: 50000


In [4]:
print(testset[0][0].shape)

torch.Size([3, 32, 32])


In [5]:
class CNN1(nn.Module):
    def __init__(self, act_func=nn.LeakyReLU):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=NUM_CHANNELS, out_channels=32, kernel_size=(3,3))
        self.af1 = act_func()
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2))

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3))
        self.af2 = act_func()
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2))

        self.fc1 = nn.Linear(in_features=2304, out_features=300)
        self.ac3 = act_func()

        self.fc2 = nn.Linear(in_features=300, out_features=NUM_CLASSES_CIFAR10) 
        self.lsfm1 = nn.LogSoftmax(dim=None)

    def forward(self, x):
        x = self.pool1(self.af1(self.conv1(x)))
        x = self.pool2(self.af2(self.conv2(x)))
        x = torch.flatten(x)
        x = self.ac3(self.fc1(x))
        x = self.lsfm1(self.fc2(x))
        return x

    def train_model(self, criterion, optimizer):
        for epoch in range(EPOCHS):
            self.train()        
            for i, (img, label) in enumerate(trainset):
                if i > 7000:
                    break
                
                label = torch.tensor(label, dtype=int)
                pred = self(img)
                loss = criterion(pred, label)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            if epoch % 5 == 0:
                print(epoch, loss.item())


    def test_model(self):
        with torch.no_grad():
            self.eval()
            correct, total = 0,0
            for i, (img, label) in enumerate(testset):
                label = torch.tensor(label, dtype=int)
                pred = self(img)
                pred = torch.argmax(pred)
                correct += int(label == pred)
                total += 1
                #print(label, pred, correct, total)

            print(f"Correct: {correct}, Total: {total}, Acc: {correct/total*100}%")

In [6]:
# Default: LeakyReLU, SGD
model = CNN1()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
model.train_model(criterion, optimizer)
model.test_model()

C:\Users\hanne\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


0 2.340357542037964
5 2.345505475997925
10 2.3605093955993652
15 2.383262872695923
20 2.397897243499756
25 2.385136604309082
30 2.333163261413574


KeyboardInterrupt: 

In [ ]:
# LeakyReLU, Adam
model2 = CNN1()
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.0001)
criterion2 = nn.CrossEntropyLoss()
model2.train_model(criterion2, optimizer2)
model2.test_model()

In [ ]:
# Tanh, SGD
model3 = CNN1(act_func=nn.Tanh)
optimizer3 = torch.optim.SGD(model3.parameters(), lr=0.0001)
criterion3 = nn.CrossEntropyLoss()
model3.train_model(criterion3, optimizer3)
model3.test_model()